In [1]:
import random
import torch
import math

In [1]:
from fol_embedding import *


In [2]:
e = Environment()
e.declare_function("f", ["A"], "A")
e.declare_function("c", [], "A")

In [3]:
e.declare_sequent("(f c) = (f x) => (f y) = c")
print(e)

(= (f c) c) => (= c c)



In [8]:
e.sequents

[([(('c', []), ('c', []))], [(('c', []), ('c', []))]),
 ([(('f', [('c', [])]), ('c', []))], [(('c', []), ('c', []))])]

In [2]:
class PlusOneNet(torch.nn.Module):
    def __init__(self):
        super(PlusOneNet, self).__init__()
        self.linear0 = torch.nn.Linear(1, 2)
        self.linear1 = torch.nn.Linear(2, 1)

    def forward(self, x):
        x = torch.nn.functional.relu(self.linear0(x))
        x = self.linear1(x)
        return x

In [3]:
class MultNet(torch.nn.Module):
    def __init__(self):
        super(MultNet, self).__init__()
        self.linear0 = torch.nn.Linear(2, 3)
        self.linear1 = torch.nn.Linear(3, 3)
        self.linear2 = torch.nn.Linear(3, 1)

    def forward(self, x):
        x = torch.nn.functional.relu(self.linear0(x))
        x = torch.nn.functional.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [4]:
class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super(DynamicNet, self).__init__()
        self.plus1 = PlusOneNet()
        self.mult = MultNet()

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        x1 = self.plus1(x)
        x2 = self.plus1(x1)
        return self.mult(torch.cat([x1, x2], dim=1))

In [5]:
# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 10000).reshape(-1, 1).to('cuda')
y = x**2 + 3*x + 2

# Construct our model by instantiating the class defined above
model = DynamicNet().to('cuda')
model.train()

DynamicNet(
  (plus1): PlusOneNet(
    (linear0): Linear(in_features=1, out_features=2, bias=True)
    (linear1): Linear(in_features=2, out_features=1, bias=True)
  )
  (mult): MultNet(
    (linear0): Linear(in_features=2, out_features=3, bias=True)
    (linear1): Linear(in_features=3, out_features=3, bias=True)
    (linear2): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [6]:

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8)

In [7]:
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

1999 476136.96875
3999 340705.6875
5999 26045.84375
7999 6750.94140625
9999 6576.740234375
11999 6477.23486328125
13999 6418.181640625
15999 6372.67578125
17999 6330.88623046875
19999 6291.73046875
21999 6254.96630859375
23999 6220.30908203125
25999 6187.57080078125
27999 6156.5673828125
29999 6127.06640625


In [11]:
model.eval()

DynamicNet(
  (plus1): PlusOneNet(
    (linear0): Linear(in_features=1, out_features=2, bias=True)
    (linear1): Linear(in_features=2, out_features=1, bias=True)
  )
  (mult): MultNet(
    (linear0): Linear(in_features=2, out_features=3, bias=True)
    (linear1): Linear(in_features=3, out_features=3, bias=True)
    (linear2): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [9]:
model(torch.tensor([[-8.0]]).to('cuda'))

tensor([[-1.7695]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
model.get_submodule('plus1')(torch.tensor([0.0]).to('cuda'))

tensor([3.3119], device='cuda:0', grad_fn=<AddBackward0>)